In [ ]:
# install botocore and boto3
# %pip install botocore boto3
# %pip install mypy-boto3-iam
# %pip install mypy-boto3-cloudformation
# %pip freeze > requirements.txt
# %pip install mypy-boto3-s3
# %pip install pydantic 


In [ ]:
from pathlib import Path
import json
from typing import Dict, List

rootpath = Path('../..')

def read_file_lines(file_path: Path) -> List[str]:
    with open(file_path, 'r') as file:
        return file.readlines()

def read_json_file(file_path: Path) -> Dict|List:
    with open(file_path, 'r') as file:
        json_content=json.load(file)
        return json_content

# len(True)

In [ ]:
request_form_lines=read_file_lines(rootpath/"ISSUE_TEMPLATE/request_prod_environment.md")

# import os
# os.linesep.join(request_form_lines)
"\n".join(request_form_lines)

In [ ]:
import re
from pydantic import BaseModel

class MdHeader(BaseModel):
    level: int=0
    title:str=None
    contents:List=[]


def get_heading_match(content:str):
    return re.match(r"^(#+) (.+)", content)


def is_heading(content:str):
    return get_heading_match(content) is not None


def title_strip(in_title:str):
    """
        strips whitespace along with last colon(:) character from title 
    """
    title=in_title.strip(":")
    last_index = title.rfind(":")  # Find the last occurrence 
    if last_index != -1:
        title = title[:last_index] + title[last_index + 1:]
    return title


def get_heading(content:str):
    heading_match=get_heading_match(content)
    if heading_match:
        return MdHeader(
            level=len(heading_match.group(1)),
            title=title_strip(heading_match.group(2))
        )
    return None


In [ ]:

from enum import Enum
import re
from pydantic import BaseModel


class ListItemType(Enum):
    Todo="list-item-todo"
    SimpleText="list-item-simple-text"
    TitleContent="list-item-title-content"

class MdListItemTodo(BaseModel):
    is_checked:bool=False
    label:str=None

class MdListItemSimpleText(BaseModel):
    text:str=None

class MdListItemTitleContent(BaseModel):
    title:str=None
    content:str=None

class MdListItem(BaseModel):
    item_type:ListItemType=None
    raw_content:str=None
    parsed_content:MdListItemTodo|MdListItemSimpleText|MdListItemTitleContent=None

class MdList(BaseModel):
    items: List[MdListItem]=[]


def get_simpletext_list_item(content:str):
    matched = re.match(r"^[-*+] (.+)", content)
    if not matched:
        return None
    return MdListItemSimpleText(
        text=matched.group(1)
    )

def get_todo_list_item(content:str):
    matched = re.match(r"^-\s+\[(x| )\]\s+(.+)", content)
    if not matched:
        return None
    return MdListItemTodo(
        is_checked=matched.group(1).strip() == "x",
        label = matched.group(2)
    )

def get_title_list_item(content:str):
    matched = re.match(r"- \*\*(.+?):\*\*\s+(.+)", content)
    if not matched:
        return None
    return MdListItemTitleContent(
        title=matched.group(1),
        content=matched.group(2)
    )

def get_list_item(content:str):
    """
        parse into Md List Item instance
    """
    if not is_list_item(content):
        return None
    list_item=MdListItem(raw_content=content)
    todo_item=get_todo_list_item(content)
    if todo_item:
        list_item.item_type=ListItemType.Todo
        list_item.parsed_content=todo_item
        return list_item
    title_item=get_title_list_item(content)
    if title_item:
        list_item.item_type=ListItemType.TitleContent
        list_item.parsed_content=title_item
        return list_item
    text_item=get_simpletext_list_item(content)
    list_item.item_type=ListItemType.SimpleText
    list_item.parsed_content=text_item
    return list_item


def is_list_item(content:str):
    return re.match(r"^[-*+] (.+)", content) is not None


In [ ]:
from enum import Enum
from pydantic import BaseModel

class AlertType(Enum):
    Important="alert-important"
    Tip="alert-tip"
    Note="alert-note"

class MdAlert(BaseModel):
    alert_type:AlertType=None
    content_lines:List[str]=[]

def get_alert_ind(content:str):
    alert_identifier_list = ["> [!IMPORTANT]", "> [!TIP]", "> [!NOTE]"]
    try:
        return alert_identifier_list.index(content.strip())
    except:
        return -1

def is_alert_type(content:str):
    return get_alert_ind(content) != -1

def get_alert(content:str):
    alert_type_list = [AlertType.Important, AlertType.Tip,AlertType.Note]
    match_ind = get_alert_ind(content)
    if match_ind != -1:
        return MdAlert(alert_type=alert_type_list[match_ind])
    return None


In [ ]:

def traverse_dict(key_depths: List[str], wrapper: Dict[str, Dict] | List[str] | str):
    if len(key_depths) == 0:
        return wrapper

    kdl = key_depths[0].lower().replace(" ", "_")
    if isinstance(wrapper, dict):
        for k, v in wrapper.items():
            kl = k.lower().replace(" ", "_")
            if kl == kdl:
                return traverse_dict(key_depths[1:], v)

    return []

class MdType(Enum):
    Header="md-header"
    ListItem="md-list-item"
    Alert="md-alert"

def get_md_type(content:str):
    """
       Supports 
        - Md Header
        - Md bullet List starts with Dash(-)
        - Md Alert
    """
    if is_heading(content):
        return MdType.Header
    if is_list_item(content):
        return MdType.ListItem
    if is_alert_type(content):
        return MdType.Alert
    return None


In [ ]:
# traverse lines,
# find the top level headers, doesn't matter which level
# if other lines don't have same level, other header becomes child of it.
# if other same level header found, it becomes sibling of list
import os
import json


def is_empty(s:str):
    return s and len(s.strip()) == 0

def parse_line_base_instance(content:str):
    content_type = get_md_type(content)
    if not content_type:
        return content
    match content_type:
        case MdType.Header:
            return get_heading(content)
        case MdType.ListItem:
            return get_list_item(content)
        case MdType.Alert:
            return get_alert(content)
        case _:
            return content

def build_list(line_list: List[str], parent_list:MdList):
    for line_num, line in enumerate(line_list):
        if is_empty(line):
            continue
        list_item=get_list_item(line)
        if not list_item:
            return line_num
        parent_list.items.append(list_item)
    return len(line_list)

def build_alert(line_list: List[str], parent_alert:MdAlert):
    for line_num, line in enumerate(line_list):
        if line.startswith("> "):
            parent_alert.content_lines.append(line[2:].strip())
        return line_num
    return len(line_list)


def build_header(line_list: List[str], parent_header:MdHeader):
    parent_header.contents = content_list = []
    # init mock
    last_content=None
    skip_lines=0
    for line_num, line in enumerate(line_list):
        if skip_lines > 0:
            skip_lines -= 1
            continue
        if not last_content:
            base_content=parse_line_base_instance(line)
            if not is_empty(line):
                last_content=base_content
                content_list.append(last_content)
                if isinstance(last_content, MdHeader):
                    skip_lines = build_header(line_list=line_list[line_num + 1:], parent_header=last_content)
        else:
            if isinstance(last_content, str):
                if is_empty(line):
                    last_content = content_list[-1] = last_content + os.linesep + line
                else:
                    base_instance=parse_line_base_instance(line)
                    if isinstance(base_instance, str):
                        last_content = content_list[-1] = last_content + os.linesep + line
                    elif isinstance(base_instance, MdHeader):
                        if base_instance.level <= parent_header.level:
                            return line_num
                        last_content=base_instance
                        content_list.append(last_content)
                        skip_lines = build_header(line_list=line_list[line_num + 1:], parent_header=last_content)
                    elif isinstance(base_instance, MdListItem):
                        last_content=MdList(items=[base_instance])
                        content_list.append(last_content)
                        skip_lines = build_list(line_list=line_list[line_num + 1:], parent_list=last_content)
                    elif isinstance(base_instance, MdAlert):
                        last_content=base_instance
                        skip_lines = build_alert(line_list=line_list[line_num + 1:], parent_alert=last_content)                        
            else:
                if is_empty(line):
                    continue
                last_content=parse_line_base_instance(line)
                if isinstance(last_content, MdHeader):
                    if last_content.level <= parent_header.level:
                        return line_num
                    content_list.append(last_content)
                    skip_lines = build_header(line_list=line_list[line_num + 1:], parent_header=last_content)
                elif isinstance(last_content, str):
                    content_list.append(last_content)
                elif isinstance(last_content, MdAlert):
                    skip_lines = build_alert(line_list=line_list[line_num + 1:], parent_alert=last_content)                        
                elif isinstance(last_content, MdListItem):
                    last_content=MdList(items=[last_content])
                    content_list.append(last_content)
                    skip_lines = build_list(line_list=line_list[line_num + 1:], parent_list=last_content)
    return len(line_list)

parsed_form=MdHeader()
lines_processed=build_header(request_form_lines, parent_header=parsed_form)
if len(request_form_lines) != lines_processed:
    raise ValueError(f"missing some lines to be processed. expected: {len(request_form_lines)}, actual: {lines_processed}")
print("parsed form into header dict: ", json.dumps(json.loads(parsed_form.json()),indent=4))


In [ ]:

def build_header(line_list: List[str], parent_header: MdHeader) -> int:
    parent_header.contents = content_list = []
    last_content = None
    skip_lines = 0

    def handle_new_content(line: str, line_num: int):
        """Handle a new line when `last_content` is None."""
        nonlocal last_content, skip_lines
        base_content = parse_line_base_instance(line)
        if not is_empty(line):
            last_content = base_content
            content_list.append(last_content)
            if isinstance(last_content, MdHeader):
                skip_lines = build_header(
                    line_list=line_list[line_num + 1:], parent_header=last_content
                )

    def handle_existing_string(line: str):
        """Handle cases where `last_content` is a string."""
        nonlocal last_content, skip_lines
        if is_empty(line):
            last_content = content_list[-1] = last_content + os.linesep + line
        else:
            base_instance = parse_line_base_instance(line)
            if isinstance(base_instance, str):
                last_content = content_list[-1] = last_content + os.linesep + line
            elif isinstance(base_instance, MdHeader):
                if base_instance.level <= parent_header.level:
                    return False
                last_content = base_instance
                content_list.append(last_content)
                skip_lines = build_header(
                    line_list=line_list[line_num + 1:], parent_header=last_content
                )
            elif isinstance(base_instance, MdListItem):
                last_content = MdList(items=[base_instance])
                content_list.append(last_content)
                skip_lines = build_list(
                    line_list=line_list[line_num + 1:], parent_list=last_content
                )
            elif isinstance(base_instance, MdAlert):
                last_content = base_instance
                skip_lines = build_alert(
                    line_list=line_list[line_num + 1:], parent_alert=last_content
                )
        return True

    def handle_non_string(line: str):
        """Handle cases where `last_content` is not a string."""
        nonlocal last_content, skip_lines
        if not is_empty(line):
            last_content = parse_line_base_instance(line)
            if isinstance(last_content, MdHeader):
                if last_content.level <= parent_header.level:
                    return False
                content_list.append(last_content)
                skip_lines = build_header(
                    line_list=line_list[line_num + 1:], parent_header=last_content
                )
            elif isinstance(last_content, str):
                content_list.append(last_content)
            elif isinstance(last_content, MdAlert):
                skip_lines = build_alert(
                    line_list=line_list[line_num + 1:], parent_alert=last_content
                )
            elif isinstance(last_content, MdListItem):
                last_content = MdList(items=[last_content])
                content_list.append(last_content)
                skip_lines = build_list(
                    line_list=line_list[line_num + 1:], parent_list=last_content
                )
        return True

    for line_num, line in enumerate(line_list):
        if skip_lines > 0:
            skip_lines -= 1
            continue
        if not last_content:
            handle_new_content(line, line_num)
        else:
            if isinstance(last_content, str):
                if not handle_existing_string(line):
                    return line_num
            else:
                if not handle_non_string(line):
                    return line_num

    return len(line_list)


parsed_form2=MdHeader()
lines_processed2=build_header(request_form_lines, parent_header=parsed_form2)
if len(request_form_lines) != lines_processed2:
    raise ValueError(f"missing some lines to be processed. expected: {len(request_form_lines)}, actual: {lines_processed2}")
print("parsed form into header dict: ", json.dumps(json.loads(parsed_form2.json()),indent=4))


In [ ]:
v1="v1.0.0"
v2="v2.0.0"
v3="v0.2.0"
v4="v0.2.0"
print(v2<v1, v1<v3, v2<v3, v3<v4)
print(v3<v2, v3<v1, v1<v2, v4<=v3)

In [ ]:
from enum import Enum

class ReqTyp(Enum):
    ReqOne="req-one"
    ReqTwo="req-two"
    ReqThree="req-three"

val="req-one"

ReqTyp(val) == ReqTyp.ReqOne

In [ ]:
"""
# Request Form to Provision

### Test Plan:

_(Provide Link to testplan issue)_

- **Regression Test Plan:** [Iss 95](https://github.com/rajexcited/personal-finance-ui/issues/95) 

### Release Details:

_(Provide details for the release or rollback)_

- **UI Version:** v0.2.1 <!-- Specify version of UI (milestone title) -->
- **API Version:** v0.2.1 <!-- Specify version of API (assigned milestone title) -->

### Environment Details:

- **Environment Name:** Test Plan Environment

### Deployment Schedule:

- **Deployment Scope:** UI and API
  _(Specify Only 1 scope value from allowed list)_

- **Preferred Date and Time:** 05-03-2025 16:38:35
  _(Use the format mm-dd-yyyy HH:MM:SS in 24-hour CST time.)_

### Additional Notes/Special Instructions:

<!-- Enter any additional notes or special instructions -->
"""

In [ ]:
from pathlib import Path

rootpath=Path("../../../")

print("rootpath=",rootpath.resolve())

def get_template_contents(relative_file_path:str):
    req_template=None

    with open(rootpath/relative_file_path, "r") as rf:
        lines=rf.readlines()
        req_template="\n".join(lines[7:])

    # print(req_prd_template)
    return req_template

In [3]:
req_nonprd=get_template_contents(".github/ISSUE_TEMPLATE/2_request_nonprod_environment.md")
req_nonprd.replace('"', '\\"')

'# Request Form to Provision\n\n\n\n<!-- OR -->\n\n\n\n# Request Form to Deprovision\n\n\n\n### Test Plan:\n\n\n\n> **Note:** _Provide Link to testplan issue_\n\n\n\n- **Regression Test Plan:** <link or NA>\n\n\n\n### Release Deployment:\n\n\n\n> **Note:** _Provide deployment versions. milestone title can be belongs to either milestone branch or master branch_\n\n\n\n- **UI Version:** v0.1.0\n\n- **API Version:** v0.1.0\n\n\n\n### Environment Details:\n\n\n\n> **Note:** _Select one option by marking an \\"X\\" in the corresponding checkbox._\n\n\n\n- [ ] Test Plan Environment\n\n- [ ] Development Environment\n\n\n\n### Deployment Schedule:\n\n\n\n- **Deployment Scope:** UI only / UI and API\n\n\n\n> **Note:** _Specify Only 1 scope value from allowed list_\n\n\n\n- **Preferred Date and Time:** 03-15-2025 13:40:35\n\n  > **Note:** _Use the format mm-dd-yyyy HH:MM:SS in 24-hour CST time._\n\n\n\n### Additional Notes/Special Instructions:\n\n\n\n> **Note:** _Enter any additional notes or s

In [8]:
import yaml

# Assuming 'config.yaml' is the name of the YAML file
try:
    with open('../../../.github/release-draft.template.yml', 'r', encoding='utf-8') as file:
        data = yaml.load(file, Loader=yaml.FullLoader)
    print(data)
except FileNotFoundError:
    print("Error: 'release-draft' not found.")
except yaml.YAMLError as e:
    print(f"Error parsing YAML file: {e}")

{'name-template': '$MILESTONE_TITLE', 'categories': [{'title': '🚀 Features', 'labels': {'include': ['feature', 'enhancement']}}, {'title': '🐛 Bug Fixes', 'labels': {'include': ['fix', 'bugfix', 'bug']}}, {'title': '🧰 Maintenance', 'labels': {'include': ['chore', 'security alert', 'javascript', 'dependencies', 'test plan']}}], 'change-template': '- $TITLE (#$ISSUE_NUMBER)', 'template': '## Changes\n\n$CHANGES\n'}
